## Preprocessing

In [167]:
import pandas as pd
import numpy as np

In [168]:
import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',999)

In [169]:
#import datasets 
work_orders = pd.read_csv('CSV_files/work_orders.csv')
OA = pd.read_csv('CSV_files/operational_assets.csv')

In [170]:
#convert dates to datetime
work_orders['FinishDate'] = pd.to_datetime(work_orders['FinishDate'])
work_orders['RaiseDate'] = pd.to_datetime(work_orders['RaiseDate'])

In [171]:
#Use only work orders from 2016 and beyond 
work_orders = work_orders[(work_orders['RaiseDate'].dt.year >= 2016.0)]

### Analysis Starts Here

In [172]:
#Create df with only failures 
WO_RaiseDate = work_orders[work_orders['WorkTypeCategory']=='Failure'].loc[:,('OperationalAssetID','OperationalAssetName','WorkTypeCategory','RaiseDate')].sort_values(['OperationalAssetName','RaiseDate'])

In [173]:
#create original WO 
WO_RaiseDate_original = WO_RaiseDate

In [174]:
#create subset of data
#WO_RaiseDate = WO_RaiseDate.head(1000)

In [175]:
#find min and max raise dates 
max_date = WO_RaiseDate['RaiseDate'].max()
min_date = WO_RaiseDate['RaiseDate'].min()


#create dataframe with max dates for each OAID 
Max_RaiseDate = WO_RaiseDate.groupby(['OperationalAssetID','OperationalAssetName'],as_index = False)['RaiseDate'].max()

In [176]:
#For all assets for which the last failed date is before the date that the data was accessed, add another row as "Non Failure"
for index, row in Max_RaiseDate.iterrows():
    if row['RaiseDate'] <= max_date:
        WO_RaiseDate = WO_RaiseDate.append({
                                            'OperationalAssetID':row['OperationalAssetID'],
                                            'OperationalAssetName':row['OperationalAssetName'],
                                            'RaiseDate':max_date,
                                            'WorkTypeCategory': 'Non Failure'
                                           }, ignore_index = True)

In [177]:
#Sort Values 
WO_RaiseDate = WO_RaiseDate.sort_values(['OperationalAssetID','RaiseDate'])

In [178]:
#Create df for maintenance 
MT = work_orders[work_orders['WorkTypeCategory'] == 'Maintenance'].loc[:,('OperationalAssetID','OperationalAssetName','FinishDate')]

In [179]:
#Create columns 
WO_RaiseDate['SinceFailure'] = 0
#Create initial date shift 
WO_RaiseDate['Raise_Date_Shifted'] = WO_RaiseDate['RaiseDate'].shift(1)

In [180]:
#Look for assets that are different 
WO_RaiseDate['DifferentAsset'] = WO_RaiseDate['OperationalAssetID'].diff()
WO_RaiseDate[WO_RaiseDate['DifferentAsset'] != 0.0]['DifferentAsset'] = WO_RaiseDate['RaiseDate']

In [181]:
#For assets that have not changed, "unshift" the dates 

min_date = WO_RaiseDate['RaiseDate'].min()
WO_RaiseDate['PreviousDate'] = 0 

for i, row in WO_RaiseDate.iterrows():
    if row['DifferentAsset'] != 0.0:
        WO_RaiseDate.loc[i,'PreviousDate'] = min_date
    else:
        WO_RaiseDate.loc[i,'PreviousDate'] = row['Raise_Date_Shifted']

In [182]:
#drop helper columns 
WO_RaiseDate = WO_RaiseDate.drop(columns = ['DifferentAsset','Raise_Date_Shifted'])

In [183]:
#Convert to datetime 
WO_RaiseDate['PreviousDate'] = pd.to_datetime(WO_RaiseDate['PreviousDate'])

In [184]:
#Calculate date since failure 
WO_RaiseDate['SinceFailure'] = WO_RaiseDate['RaiseDate'] - WO_RaiseDate['PreviousDate']

In [185]:
#create column 'Cumulative Maintenance' 
WO_RaiseDate['Cumulative_Maintenance'] = 0

In [186]:
#Calculate Cumulative Maintenance 
for ID in WO_RaiseDate['OperationalAssetID'].unique():
    for i, r in WO_RaiseDate[WO_RaiseDate['OperationalAssetID'] == ID].iterrows():
        temp = 0 
        for j, s in MT[MT['OperationalAssetID'] == ID].iterrows():
            if s['FinishDate'] <= r['RaiseDate']:
                temp = temp + 1
        WO_RaiseDate.loc[i,'Cumulative_Maintenance'] = temp 

In [187]:
# Keep only necessary columns 
WO_RaiseDate = WO_RaiseDate.loc[:,('OperationalAssetID','WorkTypeCategory','SinceFailure','OperationalAssetName','RaiseDate', 'Cumulative_Maintenance')]

In [188]:
date_delta = max_date - min_date

MA = work_orders[work_orders['WorkTypeCategory'] == 'Maintenance']
FA = work_orders[work_orders['WorkTypeCategory'] == 'Failure']

#OA maintained but not failed 
MA_not_FA = MA.merge(FA, on = 'OperationalAssetID', how = 'left', indicator = True)
MA_not_FA = MA_not_FA[MA_not_FA['_merge'] == 'left_only']['OperationalAssetID'].unique()

#Create dataframe for no failures 
No_Failure = pd.DataFrame(columns = ['OperationalAssetID','WorkTypeCategory','SinceFailure','RaiseDate'])

for i in MA_not_FA:
    No_Failure = No_Failure.append({
                                    'OperationalAssetID': i,
                                    'WorkTypeCategory':'Non Failure',
                                    'SinceFailure': date_delta,
                                    'RaiseDate':max_date        
                                    }, ignore_index=True)

#Convert OAID back to INT 
No_Failure['OperationalAssetID'] = No_Failure['OperationalAssetID'].astype('int64')

#get df with just OAID and OAName 
work_orders_OAname = work_orders.groupby('OperationalAssetID',as_index = False)['OperationalAssetName'].last()

#Get OA Name 
No_Failure = No_Failure.merge(work_orders_OAname.loc[:,('OperationalAssetID','OperationalAssetName')], how = 'left', on = 'OperationalAssetID')

### Combine Above

In [212]:
WO_RaiseDate = pd.concat([No_Failure,WO_RaiseDate])

In [217]:
WO_RaiseDate = WO_RaiseDate.reset_index()

In [218]:
#Sort Work Orders 
WO_RaiseDate = WO_RaiseDate.sort_values(['OperationalAssetID','RaiseDate'])
MT = MT.sort_values(['OperationalAssetID','FinishDate'])

#create column 'Cumulative Maintenance' 
WO_RaiseDate['Cumulative_Maintenance'] = 0

#Calculate Cumulative Maintenance 
for ID in WO_RaiseDate['OperationalAssetID'].unique():
    for i, r in WO_RaiseDate[WO_RaiseDate['OperationalAssetID'] == ID].iterrows():
        temp = 0 
        for j, s in MT[MT['OperationalAssetID'] == ID].iterrows():
            if s['FinishDate'] <= r['RaiseDate']:
                temp = temp + 1
        WO_RaiseDate.loc[i,'Cumulative_Maintenance'] = temp 

#### Get Age

In [221]:
#Create df with OAID and Year 
OA_Age = OA.loc[:,('OperationalAssetID','Year')]

In [222]:
#Drop null and unknown values in Year column 
OA_Age = OA_Age[(OA_Age['Year'] != 'UNKNOWN') & (OA['Year'] != 'UNK') & (OA['Year'].isnull() == False)]

In [223]:
#Convert year to datetime 
OA_Age['Year'] = pd.to_datetime(OA_Age['Year'])

In [224]:
#Merge columns 
WO_RaiseDate = WO_RaiseDate.merge(OA_Age, on = 'OperationalAssetID', how = 'inner')

In [225]:
#Create Age Column
WO_RaiseDate['Age'] = WO_RaiseDate['RaiseDate'] - WO_RaiseDate['Year']

In [226]:
#Create final dataset 
WO_RaiseDate_final = WO_RaiseDate.loc[:,('OperationalAssetID','WorkTypeCategory','Age','SinceFailure','OperationalAssetName','Cumulative_Maintenance')]

In [227]:
#Map WorkTeypCategory to be 0 or 1 
WO_RaiseDate_final['WorkTypeCategory'] = WO_RaiseDate_final['WorkTypeCategory'].map({'Non Failure':0,'Failure': 1})

In [228]:
WO_RaiseDate_final = WO_RaiseDate_final.rename(columns = {"WorkTypeCategory":'Fail'})

In [229]:
WO_RaiseDate_final.to_csv('Deliverable_7_SA.csv', index= False)